<a href="https://colab.research.google.com/github/fcoliveira-utfpr/IDF_equation/blob/main/calibration_IDF_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalando e importando bibliotecas

In [58]:
#Importando bibliotecas
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
from scipy import stats
from scipy.stats import (genextreme, kstest, ksone, lognorm)
from scipy.optimize import minimize
from sklearn.metrics import mean_absolute_error, mean_squared_error

##Importando e abrindo planilha

In [59]:
base_url = "https://raw.githubusercontent.com/fcoliveira-utfpr/IDF_equation/refs/heads/main/chuva_max_7801_xavier.csv"
url = base_url
df = pd.read_csv(url)
df = df.drop(columns=['index', 'Ano'])
df_1 = df.copy()
df_1.head()

,Chuva_Anahy,Chuva_Arapuã,Chuva_Ariranha do Ivaí,Chuva_Boa Esperança,Chuva_Boa Vista da Aparecida,Chuva_Campina da Lagoa,Chuva_Campo Mourão,Chuva_Capanema,Chuva_Capitão Leônidas Marques,Chuva_Céu Azul,...,Chuva_Santa Helena,Chuva_Santa Lúcia,Chuva_Santa Terezinha de Itaipu,Chuva_São José das Palmeiras,Chuva_São Miguel do Iguaçu,Chuva_São Pedro do Iguaçu,Chuva_Serranópolis do Iguaçu,Chuva_Tupãssi,Chuva_Ubiratã,Chuva_Vera Cruz do Oeste
0,76.961569,44.193844,59.932240,66.689038,73.335970,81.575968,51.994375,58.174374,80.147702,53.093042,...,117.227693,71.303437,80.806902,117.337559,71.138637,112.833027,76.247436,68.062371,86.080500,102.615430
1,97.726364,82.235168,85.311434,102.999963,134.037289,95.419165,63.667706,130.631423,135.135955,114.700760,...,114.481027,136.674088,88.552500,128.543957,114.700760,194.903407,129.422890,93.551432,82.509835,160.295416
2,98.165831,74.544503,52.104242,71.413304,148.759419,100.747697,87.014367,99.099697,148.979152,133.487956,...,109.921561,150.956751,89.815966,108.383428,127.005824,124.478891,102.395696,78.884235,111.185028,120.963159
3,95.309298,70.644237,97.726364,84.542367,60.975973,103.549296,69.490638,85.366367,62.349306,100.582897,...,69.106105,75.203703,74.819170,71.687971,69.490638,65.315705,60.756240,74.764236,79.598369,64.326906
4,77.565836,54.631175,51.005576,81.960501,46.610910,95.364231,66.249572,78.389836,43.287444,54.493841,...,73.830370,45.292510,61.305573,76.577036,57.075707,52.653575,61.305573,108.987695,78.225036,53.010642


##Desagregação da chuva máxima diária

In [60]:
##DAEE/CETESB (1980)
#Criando um dicionário com os coeficientes de desagregação
fatores = {
    '1440 min': 1.14,
    '720 min': 0.85,
    '600 min': 0.82,
    '480 min': 0.78,
    '360 min': 0.72,
    '60 min': 0.42,
    '30 min': 0.74 * 0.42,
    '20 min': 0.81 * 0.42,
    '15 min': 0.70 * 0.42,
    '10 min': 0.54 * 0.42,
    '5 min': 0.34 * 0.42
}

# Lista para armazenar os DataFrames processados
dfs = []

# Iterando sobre as colunas do DataFrame df_1
for coluna in df_1.columns:
    df_temp = df_1[[coluna]].copy()
    df_temp.rename(columns={coluna: 'Chuva (mm)'}, inplace=True)

    for duracao, fator in fatores.items():
        df_temp[duracao] = df_temp['Chuva (mm)'] * fator

    df_temp.drop(columns=['Chuva (mm)'], inplace=True)
    df_temp['Municipio'] = coluna

    dfs.append(df_temp)

# Concatenando todos os DataFrames
resultado = pd.concat(dfs, ignore_index=True)

# Exibindo o DataFrame final
# Removendo a palavra "Chuva_" da coluna "Municipio"
resultado['Municipio'] = resultado['Municipio'].str.replace('Chuva_', '', regex=False)

resultado


,1440 min,720 min,600 min,480 min,360 min,60 min,30 min,20 min,15 min,10 min,5 min,Municipio
0,87.736189,65.417334,63.108487,60.030024,55.412330,32.323859,23.919656,26.182326,22.626701,17.454884,10.990112,Anahy
1,111.408055,83.067410,80.135619,76.226564,70.362982,41.045073,30.373354,33.246509,28.731551,22.164339,13.955325,Anahy
2,111.909047,83.440956,80.495981,76.569348,70.679398,41.229649,30.509940,33.396016,28.860754,22.264010,14.018081,Anahy
3,108.652600,81.012903,78.153624,74.341253,68.622695,40.029905,29.622130,32.424223,28.020934,21.616149,13.610168,Anahy
4,88.425053,65.930960,63.603985,60.501352,55.847402,32.577651,24.107462,26.387897,22.804356,17.591932,11.076401,Anahy
...,...,...,...,...,...,...,...,...,...,...,...,...
1483,180.231815,134.383370,129.640428,123.316505,113.830620,66.401195,49.136884,53.784968,46.480836,35.856645,22.576406,Vera Cruz do Oeste
1484,93.560220,69.759813,67.297702,64.014887,59.090665,34.469555,25.507470,27.920339,24.128688,18.613559,11.719649,Vera Cruz do Oeste
1485,65.128930,48.561045,46.847125,44.561900,41.134061,23.994869,17.756203,19.435844,16.796408,12.957229,8.158255,Vera Cruz do Oeste
1486,101.075098,75.363011,72.703140,69.156646,63.836904,37.238194,27.556263,30.162937,26.066736,20.108625,12.660986,Vera Cruz do Oeste


##Calculando as intensidade máximas

In [61]:
#Critério para calcular a intensidade de aplicação
p_i = {
    "5 min": (5/60), "10 min": (10/60), "15 min": (15/60), "20 min": (20/60), "30 min": (30/60),
    "60 min": 1, "360 min": 6, "480 min": 8, "600 min": 10, "720 min": 12,
    "1440 min": 24
}

In [62]:
# Criar um novo DataFrame i_max
i_max = resultado.copy()
municipios = i_max['Municipio']
i_max = i_max.apply(lambda x: pd.to_numeric(x, errors = 'coerce'), axis=1)
i_max.drop(columns=['Municipio'], inplace=True)

# Dividir cada coluna pelo valor correspondente no dicionário p_i
for coluna in i_max.columns:
    i_max[coluna] = i_max[coluna] / p_i[coluna]

# Renomear as colunas para refletir a operação realizada
i_max.columns = [f"i_max_{col}" for col in i_max.columns]
i_max['Municipio'] = municipios
i_max

,i_max_1440 min,i_max_720 min,i_max_600 min,i_max_480 min,i_max_360 min,i_max_60 min,i_max_30 min,i_max_20 min,i_max_15 min,i_max_10 min,i_max_5 min,Municipio
0,3.655675,5.451444,6.310849,7.503753,9.235388,32.323859,47.839311,78.546978,90.506805,104.729303,131.881345,Anahy
1,4.642002,6.922284,8.013562,9.528321,11.727164,41.045073,60.746708,99.739527,114.926204,132.986036,167.463898,Anahy
2,4.662877,6.953413,8.049598,9.571169,11.779900,41.229649,61.019880,100.188047,115.443017,133.584063,168.216968,Anahy
3,4.527192,6.751075,7.815362,9.292657,11.437116,40.029905,59.244260,97.272670,112.083735,129.696893,163.322013,Anahy
4,3.684377,5.494247,6.360399,7.562669,9.307900,32.577651,48.214924,79.163692,91.217423,105.551589,132.916816,Anahy
...,...,...,...,...,...,...,...,...,...,...,...,...
1483,7.509659,11.198614,12.964043,15.414563,18.971770,66.401195,98.273768,161.354903,185.923345,215.139871,270.916875,Vera Cruz do Oeste
1484,3.898342,5.813318,6.729770,8.001861,9.848444,34.469555,51.014941,83.761018,96.514753,111.681357,140.635783,Vera Cruz do Oeste
1485,2.713705,4.046754,4.684713,5.570237,6.855677,23.994869,35.512406,58.307532,67.185634,77.743376,97.899066,Vera Cruz do Oeste
1486,4.211462,6.280251,7.270314,8.644581,10.639484,37.238194,55.112527,90.488811,104.266943,120.651748,151.931831,Vera Cruz do Oeste


##Avaliar a aderências das FDP aos dados

In [63]:
# Função para ajustar as distribuições e retornar os resultados
def ajustar_distribuicoes(chuva_anual, nome_serie):
    # Crie um intervalo de valores para a precipitação
    x = np.linspace(0, chuva_anual[~pd.isnull(chuva_anual)].astype(float).max(), 1000)

    # Função para ajustar a distribuição Log-Pearson III
    def log_pearson3_fit(data):
        log_data = np.log(data)
        shape, loc, scale = stats.pearson3.fit(log_data)
        return shape, loc, scale

    # Ajuste das distribuições
    ## Gumbel (Extreme Value Type I)
    params_gumbel = stats.gumbel_r.fit(chuva_anual)
    D_gumbel, p_value_gumbel = kstest(chuva_anual, 'gumbel_r', args=params_gumbel)

    ## Log-Normal
    params_ln = stats.lognorm.fit(chuva_anual)
    D_ln, p_value_ln = kstest(chuva_anual, 'lognorm', args=params_ln)

    ## GEV (Generalized Extreme Value)
    params_gev = genextreme.fit(chuva_anual)
    D_gev, p_value_gev = kstest(chuva_anual, 'genextreme', args=params_gev)

    ## Log-Pearson III
    shape_lp3, loc_lp3, scale_lp3 = log_pearson3_fit(chuva_anual)
    D_lp3, p_value_lp3 = kstest(np.log(chuva_anual), 'pearson3', args=(shape_lp3, loc_lp3, scale_lp3))

    # Cálculo do D crítico
    n = len(chuva_anual)
    alpha = 0.05  # Nível de significância
    D_critical = ksone.ppf(1 - alpha / 2, n)

    # Crie um dicionário com os dados
    data = {
        "Série": [nome_serie] * 4,  # Repete o nome da série para cada distribuição
        "Distribuição": ["Gumbel", "Log-Normal", "GEV", "Log-Pearson III"],
        "P-valor": [p_value_gumbel, p_value_ln, p_value_gev, p_value_lp3],
        "Dsup": [D_gumbel, D_ln, D_gev, D_lp3],
        "Dcrítico": [D_critical] * 4,  # Valor de D calculado (Dsup) tem que ser menor que D crítico.
        "Coeficientes": [
            f"loc={params_gumbel[0]:.1f}, scale={params_gumbel[1]:.1f}",
            f"shape={params_ln[0]:.1f}, loc={params_ln[1]:.1f}, scale={params_ln[2]:.1f}",
            f"c={params_gev[0]:.1f}, loc={params_gev[1]:.1f}, scale={params_gev[2]:.1f}",
            f"shape={shape_lp3:.1f}, loc={loc_lp3:.1f}, scale={scale_lp3:.1f}",
        ]
    }

    # Crie o DataFrame
    result = pd.DataFrame(data)

    # Determine o melhor ajuste com base no nível de significância
    result["Melhor Ajuste"] = "Não significativo"  # Começa com "Não significativo" e é atualizado se um ajuste for adequado
    result.loc[result["P-valor"] > alpha, "Melhor Ajuste"] = result.loc[result["P-valor"] > alpha, "Distribuição"]

    # Ordene os resultados pelo Dsup
    result = result.sort_values(by='Dsup').reset_index(drop=True)
    result = result[['Série', 'Melhor Ajuste', 'P-valor', 'Dsup', 'Dcrítico', 'Coeficientes']]

    return result

# Função para ajustar a distribuição Log-Pearson III
def log_pearson3_fit(data):
    log_data = np.log(data)
    shape, loc, scale = stats.pearson3.fit(log_data)
    return shape, loc, scale

In [67]:
# Suponha que você tenha uma lista de cidades
cidades = municipios.unique()

# DataFrame final que irá armazenar os resultados de todas as cidades
resultados_finais_todas_cidades = pd.DataFrame()

# Iterar sobre cada cidade na lista
for cidade in cidades:
    # Filtrar o DataFrame i_max para a cidade atual
    df_max = i_max[i_max['Municipio'] == cidade].copy()

    # Remover a coluna 'Municipio' pois não será mais necessária
    df_max.drop(columns=['Municipio'], inplace=True)

    # DataFrame temporário para armazenar os resultados da cidade atual
    resultados_finais = pd.DataFrame()

    # Aplicar a função para cada coluna do df_max
    for coluna in df_max.columns:
        chuva_anual = df_max[coluna].values
        resultado = ajustar_distribuicoes(chuva_anual, coluna)
        resultados_finais = pd.concat([resultados_finais, resultado], ignore_index=True)

    # Adicionar a coluna 'Municipio' ao DataFrame de resultados da cidade atual
    resultados_finais['Municipio'] = cidade

    # Concatenar os resultados da cidade atual ao DataFrame final de todas as cidades
    resultados_finais_todas_cidades = pd.concat([resultados_finais_todas_cidades, resultados_finais], ignore_index=True)

# Exibir o DataFrame final com os resultados de todas as cidades
resultados_finais_todas_cidades

,Série,Melhor Ajuste,P-valor,Dsup,Dcrítico,Coeficientes,Municipio
0,i_max_1440 min,Log-Normal,0.771094,0.114273,0.237884,"shape=0.2, loc=0.2, scale=3.9",Anahy
1,i_max_1440 min,Log-Pearson III,0.762821,0.115197,0.237884,"shape=0.1, loc=1.4, scale=0.2",Anahy
2,i_max_1440 min,GEV,0.752859,0.116300,0.237884,"c=0.1, loc=3.7, scale=0.9",Anahy
3,i_max_1440 min,Gumbel,0.650195,0.127306,0.237884,"loc=3.7, scale=0.9",Anahy
4,i_max_720 min,Log-Normal,0.771094,0.114273,0.237884,"shape=0.2, loc=0.2, scale=5.8",Anahy
...,...,...,...,...,...,...,...
2107,i_max_10 min,Log-Normal,0.874314,0.101608,0.237884,"shape=0.4, loc=33.7, scale=96.0",Vera Cruz do Oeste
2108,i_max_5 min,Gumbel,0.957527,0.086848,0.237884,"loc=148.9, scale=43.5",Vera Cruz do Oeste
2109,i_max_5 min,GEV,0.911219,0.096046,0.237884,"c=-0.0, loc=147.8, scale=42.8",Vera Cruz do Oeste
2110,i_max_5 min,Log-Pearson III,0.892993,0.098910,0.237884,"shape=0.3, loc=5.1, scale=0.3",Vera Cruz do Oeste
